In [19]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from scipy.optimize import curve_fit
import datetime
from matplotlib.dates import date2num
import numpy as np
import plotly
import chart_studio
import chart_studio.plotly as py
from plotly.offline import plot, iplot
chart_studio.tools.set_credentials_file(username='sorin.bolos', api_key='1VPkxbMHak7Y7M9dyDQD')

pacienti = pd.read_csv('pacienti_agg.csv', parse_dates=[0], dayfirst=True)
predictii = pd.read_csv('predictii_cumulat.csv', parse_dates=[0], dayfirst=True)

In [25]:
#Total cazuri
total = pacienti.merge(predictii, left_on='data', right_on='data', how='outer')
x = total['data']
y1 = total['cumulat']
y2 = total['t1']

fig = go.Figure(data=[
    go.Bar(name='Predictie', x=x, y=y2),
    go.Bar(name='Real', x=x, y=y1)
])

fig.update_layout(title="Numar total de cazuri")

fig.show()
#py.iplot(fig, filename="total_cazuri")
#pio.write_html(fig, file='infectii_totale.html', auto_open=True)

In [26]:
#Cazuri externe pe zi
x = total['data']
y = total['pe_zi']

fig = go.Figure(data=[
    go.Bar(x=x, y=y)
])
fig.update_layout(title="Cazuri noi pe zi")
fig.show()

#py.iplot(fig, filename="cazuri_noi_pe_zi")
#pio.write_html(fig, file='infectii_noi_zi_externe.html', auto_open=True)

In [27]:
def sigmoid(x, x0,y0,c,k):
    return c / (1 + np.exp(-k*(x-x0))) + y0

def exponential(x, a, b, c):
    return a * np.exp(b * x) + c

forward = 14

l = len(pacienti)
x = np.linspace(1, l, l)
y = list(i/1000 for i in pacienti["cumulat"])

popt, pcov = curve_fit(sigmoid, x, y)
#popt_e, pcov_e = curve_fit(exponential, x, y)

yp = pacienti['cumulat']

x2 = np.linspace(1, l+forward, l+forward)
date = pd.date_range(start='26-02-2020', periods=l+forward)

fig = go.Figure(data=[
    go.Scatter(name='Predictie', x=date, y=1000*sigmoid(x2, *popt), mode='lines+markers'),
    #go.Scatter(name='Predictie pesimista', x=date, y=1000*exponential(x2, *popt_e), mode='lines+markers'),
    go.Bar(name='Real', x=date, y=yp)
])
fig.update_layout(title="Cazuri totale + predictie curenta")
fig.show()

#py.iplot(fig, filename="predictie_curenta")

In [28]:
date = pd.date_range(start='26-02-2020', periods=l+forward)
x2 = np.linspace(1, l+forward, l+forward)

figure_data = []
for i in range(18, l+1):
    workset = pacienti[:i]
    l_2 = len(workset)
    x_2 = np.linspace(1, l_2, l_2)
    y_2 = list(i/1000 for i in workset["cumulat"])
    popt_2, pcov_2 = curve_fit(sigmoid, x_2, y_2)
    figure_data.append(go.Scatter(name=f'Predictie {date[i].day} Martie', x=date, y=1000*sigmoid(x2, *popt_2)),)

fig = go.Figure(data=figure_data)
fig.update_layout(title="Evolutia predictiilor")
#fig.show()

py.iplot(fig, filename="evolutie_predictii")

In [32]:
l = len(pacienti)
x = np.linspace(1, l, l)
y = list(i/1000 for i in pacienti["cumulat"])

popt_e, pcov_e = curve_fit(exponential, x, y)

yp = pacienti['cumulat']

x2 = np.linspace(1, l+forward, l+forward)
date = pd.date_range(start='26-02-2020', periods=l+forward)

fig = go.Figure(data=[
    go.Scatter(name='Predictie pesimista', x=date, y=1000*exponential(x2, *popt_e), mode='lines+markers'),
    go.Bar(name='Real', x=date, y=yp)
])
fig.update_layout(title="Cazuri totale + predictie pesimista")
fig.show()

#py.iplot(fig, filename="predictie_pesimista")

In [31]:
date = pd.date_range(start='26-02-2020', periods=l+forward)
x2 = np.linspace(1, l+forward, l+forward)

figure_data = []
for i in range(18, l+1):
    workset = pacienti[:i]
    l_2 = len(workset)
    x_2 = np.linspace(1, l_2, l_2)
    y_2 = list(i/1000 for i in workset["cumulat"])
    popt_2, pcov_2 = curve_fit(exponential, x_2, y_2)
    figure_data.append(go.Scatter(name=f'Predictie {date[i].day} Martie', x=date, y=1000*exponential(x2, *popt_2)),)

fig = go.Figure(data=figure_data)
fig.update_layout(title="Evolutia predictiilor pesimiste")
fig.show()

#py.iplot(fig, filename="evolutie_predictii_pesimiste")

In [34]:
reference_point = 20
reference_workset = pacienti[:reference_point]
reference_l = len(reference_workset)
x_r = np.linspace(1, reference_l, reference_l)
y_r = list(i/1000 for i in reference_workset["cumulat"])

popt_realistic, pcov_realistic = curve_fit(sigmoid, x_r, y_r)
popt_pesimistic, pcov_pesimistic = curve_fit(exponential, x_r, y_r)

date_reference = pd.date_range(start='26-02-2020', periods=reference_l+forward)
x_reference = np.linspace(1, reference_l+forward, reference_l+forward)

fig = go.Figure(data=[
    go.Scatter(name='Predictie realista', x=date_reference, y=1000*sigmoid(x_reference, *popt_realistic), mode='lines+markers'),
    go.Scatter(name='Predictie pesimista', x=date_reference, y=1000*exponential(x_reference, *popt_pesimistic), mode='lines+markers'),
    #go.Bar(name='Real', x=date, y=yp)
])
fig.update_layout(title="Pesimist vs realist")
fig.show()
